In [1]:
from asim.dataset.scene.scene_builder import ArrowSceneBuilder
from asim.dataset.scene.scene_filter import SceneFilter

In [2]:


from nuplan.planning.utils.multithreading.worker_sequential import Sequential
# from nuplan.planning.utils.multithreading.worker_ray import RayDistributed

In [4]:
import os, psutil

def print_memory_usage():
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"Memory usage: {memory_info.rss / 1024 ** 2:.2f} MB")


split = "nuplan_mini_val"
scene_filter = SceneFilter(split_names=[split])
scene_builder = ArrowSceneBuilder("/home/daniel/asim_workspace/data")
worker = Sequential()  
# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)

len(scenes)

TypeError: LogMetadata.__init__() got an unexpected keyword argument 'timestep_seconds'

In [ ]:
from pathlib import Path
from typing import Optional, Tuple

import matplotlib.animation as animation
import matplotlib.pyplot as plt
from tqdm import tqdm

from asim.common.geometry.base import Point2D, StateSE2
from asim.common.geometry.bounding_box.bounding_box import BoundingBoxSE2
from asim.common.visualization.color.default import EGO_VEHICLE_CONFIG
from asim.common.visualization.matplotlib.observation import (
    add_bounding_box_to_ax,
    add_box_detections_to_ax,
    add_default_map_on_ax,
    add_traffic_lights_to_ax,
)
from asim.dataset.arrow.conversion import TrafficLightDetectionWrapper
from asim.dataset.maps.abstract_map import AbstractMap
from asim.dataset.observation.detection.detection import BoxDetectionWrapper
from asim.dataset.scene.abstract_scene import AbstractScene
from nuplan.common.actor_state.ego_state import EgoState
import io
from PIL import Image


def add_ego_vehicle_to_ax_(ax: plt.Axes, ego_state: EgoState) -> None:
    bounding_box = BoundingBoxSE2(
        center=StateSE2(*ego_state.center),
        length=ego_state.car_footprint.length,
        width=ego_state.car_footprint.width,
    )
    add_bounding_box_to_ax(ax, bounding_box, EGO_VEHICLE_CONFIG)


def _plot_scene_on_ax(
    ax: plt.Axes,
    map_api: AbstractMap,
    ego_state: EgoState,
    box_detections: BoxDetectionWrapper,
    traffic_light_detections: TrafficLightDetectionWrapper,
    radius: float = 80,
) -> plt.Axes:

    point_2d = Point2D(ego_state.center.x, ego_state.center.y)
    add_default_map_on_ax(ax, map_api, point_2d, radius=radius)
    add_traffic_lights_to_ax(ax, traffic_light_detections, map_api)

    add_box_detections_to_ax(ax, box_detections)
    add_ego_vehicle_to_ax_(ax, ego_state)

    ax.set_xlim(point_2d.x - radius, point_2d.x + radius)
    ax.set_ylim(point_2d.y - radius, point_2d.y + radius)


    ax.set_aspect("equal", adjustable="box")
    return ax

def plot_scene_to_image(
    map_api: AbstractMap,
    ego_state: EgoState,
    box_detections: BoxDetectionWrapper,
    traffic_light_detections: TrafficLightDetectionWrapper,
    radius: float = 80,
    figsize: Tuple[int, int] = (8, 8),
) -> Image:

    fig, ax = plt.subplots(figsize=figsize)
    _plot_scene_on_ax(ax, map_api, ego_state, box_detections, traffic_light_detections, radius)
    ax.set_aspect("equal", adjustable="box")
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
    # plt.tight_layout()
    
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    plt.close(fig)
    buf.seek(0)
    img = Image.open(buf)
    return img


In [5]:
from asim.simulation.gym.demo_gym_env import DemoGymEnv

images = []
action = [1.0, 0.1]  # Placeholder action, replace with actual action logic
env = DemoGymEnv(scenes)
map_api, ego_state, detection_observation = env.reset()

images.append(
    plot_scene_to_image(
        map_api,
        ego_state,
        detection_observation.box_detections,
        detection_observation.traffic_light_detections,
    )
)

for i in range(150):
    ego_state, detection_observation, end = env.step(action)
    images.append(
        plot_scene_to_image(
            map_api,
            ego_state,
            detection_observation.box_detections,
            detection_observation.traffic_light_detections,
        )
    )
    if end:
        print("End of scene reached.")
        break



In [6]:
import numpy as np


def create_gif(images, output_path, duration=100):
    """
    Create a GIF from a list of PIL images.

    Args:
        images (list): List of PIL.Image objects.
        output_path (str): Path to save the GIF.
        duration (int): Duration between frames in milliseconds.
    """
    if images:
        print(len(images))
        images_p = [img.convert('P', palette=Image.ADAPTIVE) for img in images]
        images_p[0].save(
            output_path,
            save_all=True,
            append_images=images_p[1:],
            duration=duration,
            loop=0
        )

create_gif(images, f"{split}_{np.random.randint(0, 1000)}_{action}.gif", duration=20)

151
